**_Purpose: Read Silver → aggregate → save as Gold._**

**Gold Layer – Aggregated & Analytics-Ready Data**

The Gold layer is where we create business-level insights. Here, you’ll do things like:

Daily summaries (average temperature, total precipitation, average humidity)

Hourly trends or peaks

Optional: Detect extreme weather conditions or alerts

In [0]:
# Load Silver table into a DataFrame
silver_df = spark.table("weather_silver")


from pyspark.sql.functions import col, to_date, avg, sum

# Convert hour to date
gold_df = silver_df.withColumn("date", to_date(col("hour")))

# Aggregate by date
daily_summary = gold_df.groupBy("date").agg(
    avg("temperature_2m").alias("avg_temp"),
    avg("relative_humidity_2m").alias("avg_humidity"),
    sum("precipitation").alias("total_precipitation")
)

daily_summary.show()


+----------+------------------+------------------+-------------------+
|      date|          avg_temp|      avg_humidity|total_precipitation|
+----------+------------------+------------------+-------------------+
|2025-09-23|22.195833206176758|            75.375|  8.800000011920929|
|2025-09-24|21.637500047683716| 85.66666666666667| 1.2000000178813934|
|2025-09-22|20.747999992370605|             73.32|                0.0|
|2025-09-25|24.199999968210857|            79.625|  7.999999910593033|
|2025-09-28| 23.24999992052714|53.041666666666664|                0.0|
|2025-09-27|22.733333428700764|60.916666666666664|                0.0|
|2025-09-26| 23.12083339691162| 78.70833333333333|   14.2000000923872|
+----------+------------------+------------------+-------------------+



**save this Gold layer as a Delta table**

In [0]:

# Sort by date
daily_summary_sorted = daily_summary.orderBy("date")



# Load existing Gold table if exists
try:
    existing_gold = spark.table("weather_gold")
    # Combine old and new, remove duplicates by date
    combined_gold = existing_gold.union(daily_summary_sorted).dropDuplicates(["date"])
except Exception:
    # If Gold table doesn't exist yet
    combined_gold = daily_summary_sorted

# Save / Overwrite Gold table with merged data
combined_gold.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("weather_gold")



# Quick check
spark.sql("SELECT * FROM weather_gold").show()


+----------+------------------+------------------+-------------------+
|      date|          avg_temp|      avg_humidity|total_precipitation|
+----------+------------------+------------------+-------------------+
|2025-09-23|22.195833206176758|            75.375|  8.800000011920929|
|2025-09-24|21.637500047683716| 85.66666666666667| 1.2000000178813934|
|2025-09-22|20.747999992370605|             73.32|                0.0|
|2025-09-25|24.199999968210857|            79.625|  7.999999910593033|
|2025-09-28| 23.24999992052714|53.041666666666664|                0.0|
|2025-09-27|22.733333428700764|60.916666666666664|                0.0|
|2025-09-26| 23.12083339691162| 78.70833333333333|   14.2000000923872|
+----------+------------------+------------------+-------------------+

